# 베이스라인 모델: LightGBM (최종 제출용)

이 노트북은 전처리가 완벽하게 완료된 데이터(train_final.csv, test_final.csv)를 사용하여,
가장 기본적인 LightGBM 모델을 학습하고 리더보드 제출 파일을 생성합니다.

### 목표
1. 전처리된 데이터 로드 및 품질 확인 (행 개수 검증)
2. LightGBM 모델 학습 (기본 하이퍼파라미터 사용)
3. 검증 데이터(Validation)에 대한 RMSE 평가
4. 정수형(Integer) 예측값이 담긴 제출 파일 생성

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# ==========================================
# 1. 데이터 경로 자동 탐지
# ==========================================
# 다양한 실행 환경(로컬, 서버, 코랩 등)에서 유연하게 데이터를 찾기 위해 여러 경로를 리스트로 관리합니다.
POSSIBLE_PATHS = [
    'data/raw/processed',      # 1순위: run_pipeline_gpu.ipynb가 저장한 기본 경로
    '../data/processed',       # 상위 폴더의 processed
    'data/processed',          # 현재 폴더 내 processed
    './processed',             # 현재 폴더 내 processed (상대 경로)
    './'                       # 현재 폴더에 csv가 바로 있는 경우
]

DATA_DIR = None
for path in POSSIBLE_PATHS:
    # 해당 경로에 학습 데이터 파일이 실제로 존재하는지 확인합니다.
    if os.path.exists(os.path.join(path, 'train_final.csv')):
        DATA_DIR = path
        break

# 만약 모든 경로를 다 뒤져도 파일이 없다면 에러를 발생시켜 실행을 중단합니다.
if DATA_DIR is None:
    print(f"현재 작업 경로: {os.getcwd()}")
    raise FileNotFoundError("전처리된 데이터 'train_final.csv'를 찾을 수 없습니다. 전처리 파이프라인(run_pipeline_gpu.ipynb)을 먼저 실행해주세요.")

print(f"데이터 디렉토리 확인됨: {DATA_DIR}")
TRAIN_PATH = os.path.join(DATA_DIR, 'train_final.csv')
TEST_PATH = os.path.join(DATA_DIR, 'test_final.csv')

# 제출 양식 파일(sample_submission.csv) 찾기
# 보통 processed가 아닌 raw 폴더(상위 폴더)에 원본 데이터와 함께 있습니다.
RAW_DIR = os.path.dirname(DATA_DIR)
SUBMISSION_PATH = os.path.join(RAW_DIR, 'sample_submission.csv')

# 만약 바로 위에서 못 찾으면 한 번 더 상위로 가거나 다른 경로를 시도합니다.
if not os.path.exists(SUBMISSION_PATH):
    # 예: data/raw/processed -> data/raw/sample_submission.csv
    SUBMISSION_PATH = os.path.join(RAW_DIR, '../sample_submission.csv')

print(f"학습 데이터: {TRAIN_PATH}")
print(f"테스트 데이터: {TEST_PATH}")
print(f"제출 양식: {SUBMISSION_PATH}")

## 2. 데이터 로드 및 전처리 (모델링용)

In [ ]:
# CSV 파일 로드
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print(f"Train Shape: {train.shape}")
print(f"Test Shape: {test.shape}")

# [중요] Test 데이터 개수 검증 (9272개)
# 전처리 과정에서 테스트 데이터의 행이 삭제되었는지 확인하는 안전장치입니다.
if test.shape[0] != 9272:
    print("경고: Test 데이터 개수가 9272개가 아닙니다! 제출 시 에러가 발생할 수 있습니다.")
else:
    print("Test 데이터 개수가 정상입니다 (9272개).")

# Target(타겟) 변수 분리 및 로그 변환
# 집값과 같은 금액 데이터는 분포의 꼬리가 긴(Skewed) 형태를 띠므로 로그 변환(log1p)을 해주는 것이 학습에 유리합니다.
if 'target' in train.columns:
    y = np.log1p(train['target'])
    X = train.drop(columns=['target'])
else:
    raise ValueError("Train 데이터에 예측해야 할 'target' 컬럼이 없습니다.")

# Test 데이터에는 target 컬럼이 없으므로 그대로 사용합니다.
X_test = test.copy()
# 혹시라도 실수로 target이 포함되어 있다면 제거합니다.
if 'target' in X_test.columns:
    X_test.drop(columns=['target'], inplace=True)

# 범주형(String/Object) 변수 라벨 인코딩 Process
# 머신러닝 모델은 문자열을 이해하지 못하므로 숫자로 변환해주어야 합니다.
cat_cols = X.select_dtypes(include=['object']).columns
print(f"범주형 컬럼({len(cat_cols)}개): {list(cat_cols)}")

for col in cat_cols:
    le = LabelEncoder()
    
    # 학습 데이터와 테스트 데이터에 존재하는 모든 범주(Category)를 학습시키기 위해 합칩니다.
    # 일부 값은 학습엔 없는데 테스트에만 있거나 그 반대의 경우가 있을 수 있기 때문입니다.
    full_data = pd.concat([X[col], X_test[col]], axis=0).astype(str)
    le.fit(full_data)
    
    # 숫자로 변환 적용
    X[col] = le.transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

print("라벨 인코딩 완료.")

## 3. 학습/검증 데이터 분리

In [ ]:
# 전체 학습 데이터를 모델 학습용(Training)과 성능 평가용(Validation)으로 8:2 비율로 나눕니다.
# random_state=42로 고정하여 실행할 때마다 결과가 달라지지 않도록 합니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"학습셋 크기: {X_train.shape}")
print(f"검증셋 크기: {X_val.shape}")

## 4. LightGBM 모델 학습 (Baseline)

In [ ]:
# LightGBM 모델 객체 생성 (기본 설정 사용)
# n_jobs=-1: 모든 CPU 코어를 사용하여 학습 속도를 높입니다.
model = lgb.LGBMRegressor(random_state=42, n_jobs=-1)

print("모델 학습 시작...")
# 학습 데이터(X_train)와 정답(y_train)을 넣어 모델을 학습시킵니다.
model.fit(X_train, y_train)
print("학습 완료.")

## 5. 평가 (RMSE) 및 시각화

In [ ]:
# 검증 데이터(X_val)에 대해 예측을 수행합니다.
# 결과는 로그 변환된 값이므로 실제 가격으로 보기 어렵습니다.
pred_val_log = model.predict(X_val)

# 로그 변환된 값을 다시 원래 스케일(실제 가격)로 복원합니다. (np.expm1 사용)
pred_val = np.expm1(pred_val_log)
actual_val = np.expm1(y_val)

# 성능 지표인 RMSE(평균조화오차)를 계산합니다.
rmse = np.sqrt(mean_squared_error(actual_val, pred_val))
print(f"=========================================")
print(f"Baseline 모델 검증 RMSE: {rmse:,.0f}")
print(f"=========================================")

# 변수 중요도(Feature Importance) 시각화
# 한글 폰트 깨짐 문제를 방지하기 위해 주요 변수명을 영어로 매핑하여 보여줍니다.
imp_df = pd.DataFrame({
    'Feature': model.feature_name_,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False).head(20)

# 한글 -> 영어 매핑 딕셔너리
name_map = {
    '좌표X': 'Coord X', 
    '좌표Y': 'Coord Y', 
    '건축년도': 'Year Built', 
    '주차대수': 'Parking Count',
    'k-전체세대수': 'Total Households', 
    'k-연면적': 'Total Floor Area', 
    'k-주거전용면적': 'Residental Area', 
    '동': 'Dong', 
    '구': 'Gu',
    '계약일자': 'Contract Date', 
    '층': 'Floor'
}
# 딕셔너리에 없는 이름은 원래 이름 그대로 표시합니다.
imp_df['Feature_En'] = imp_df['Feature'].map(lambda x: name_map.get(x, x))

plt.figure(figsize=(10, 8))
sns.barplot(data=imp_df, x='Importance', y='Feature_En', palette='viridis')
plt.title("Feature Importance (Top 20)")
plt.show()

## 6. 제출 파일 생성 (최종)

In [ ]:
# 테스트 셋(X_test)에 대해 최종 예측을 수행합니다.
pred_test_log = model.predict(X_test)
pred_test_float = np.expm1(pred_test_log)

# [중요] 대회 규칙이나 통상적인 관례에 따라 예측값을 정수(Integer)로 변환합니다. (반올림)
pred_test_int = np.round(pred_test_float).astype(int)

# 제출 파일 생성 프로세스
if os.path.exists(SUBMISSION_PATH):
    submission = pd.read_csv(SUBMISSION_PATH)
    
    # 데이터 개수가 맞는지 마지막으로 한 번 더 확인합니다.
    if len(submission) != len(pred_test_int):
        print(f"[치명적 오류] 예측 개수({len(pred_test_int)})와 제출 양식 개수({len(submission)})가 다릅니다.")
    else:
        # 예측값을 'target' 컬럼에 덮어씁니다.
        submission['target'] = pred_test_int
        
        # 결과 저장
        save_path = os.path.join(DATA_DIR, 'submission_baseline.csv')
        submission.to_csv(save_path, index=False)

        print(f"제출 파일 저장 완료: {save_path}")
        print("생성된 파일을 다운로드하여 리더보드에 제출하세요!")
        print(submission.head())
else:
    print("sample_submission.csv를 찾지 못했습니다. 경로를 확인해주세요.")